# Sandbox for development

game_generator presentation, requires "PyYAML" library to save/load game definition in concise, human-readable way

In [1]:
import game_generator

In [2]:
game_generator.generatePlayable(outputDir="generated/games", n=20) #will generate files game_0.yml to game_20.yml in directory "generated/games"

0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%


generated games are also tested, 3 conditions have to be met in 20 random playouts to pass a game as initially 'playable':
  - each of two players have to win at least once
  - no game can end in less than 8 rounds
  - every piece on board must at least once be able to move during all playouts

## Generated space

### Searchable space
- dimension of table, rectangular
- types of pawns
  - possible moves
  - possible attack moves
- initial placement of pawns
  - not scattered randomly
- winning condition
  - could specify a single target square that pawns have to reach
  - could specify multiple targets that pawns have to reach (maybe at the same time)
  - specific piece type have to be removed to win
  - an amount of enemy pieces left have to be reached

### Game elements

- game definition:
  - initial placement of pieces
  - definition of types of pieces

- piece type:
  - holds its all possible move definitions

- move definition:
  - 3 types, hop, leap, slide 
  - defines movement relative to its piece
  - defines conditions for the move (e.g. only to empty square or have to jump over an enemy)

- move conditions:
  - only allow to make a move when specific scenario is met

- board:
  - rectangular (with x,y coordinates)
  - enumerable
  - keeps track all pieces and their owners

- game state:
  - current position of all pieces in the game
  - has all information needed to define all legal moves for the next turn

- win conditions:
  - defines condition that have to be met in a game state for a specific player to lose e.g:
    - win if a piece reaches the other side of the board
    - win if a specific enemy piece is removed from the board
    - win if 2 or more specified squares on the boards are taken by a player

## Example of generated game

```yaml
dimensions: 7x5
pieces:
- '1':
    moves:
    - canAttack: true
      hop: FFR
    - leap: FR
      if: origin facing L ally
    - slide: R
placement: 1111110 1100011
player2mirrored: true
winConditions:
  piecePlacedAt:
    3: 0,1,2
```

This specific game is played on a board with 7 columns and 5 rows,
it has only one piece, with 4 possible moves, each of all 3 types:
  - hop: FFR, means that a piece can move 'Forward Forward Right' from its current position, 'canAttack: true' means that the move can end on an enemy piece, which causes the enemy piece to be removed
  - leap: FR, means that a piece can 'leap' over a square that is 'Forward Right' from its current position, landing on a square behind it. Additionaly leaping over an enemy piece causes this enemy to be removed. This move also comes with condition 'if: origin facing L ally'. Origin means that this condition is checked relative to pieces current position, if it was "Destination" it would be checked against the square that the move ends on. Next part of this condition 'facing L ally'] means that the first piece on the left of current position has to be a piece of the same player. If this condition is not satisfied, the move cannot be made
  - slide: R, means that a piece can 'slide' in 'Right' direction, effectively being able to land on any piece on it's right side, until it reaches border of the board, hits an ally or lands on an enemy.
  
"placement: 1111110 1100011" describes how the pieces are distributed across the board. Here there are two rows, space-separated. Zero means that no piece is placed on a square, other numbers represent piece identifiers.
player2mirrored is a field that tells whether enemy pieces should be mirrored along the x axis when they are placed on the opposite side of the board
winConditions represents all conditions that have to be met for a game to end with a winner
piecePlacedAt... means that any piece from player 1 have to land on 3rd row on any of these columns (0,1 or 2)

In [6]:
import yaml
import utils
game_path = "./generated/games/game_10.yml"
game = utils.fromFile(game_path)

In [7]:
for line in yaml.dump(game.toDict()).split("\n"):
    print(line)

dimensions: 8x5
pieces:
- '3':
    moves:
    - leap: FR
- '3':
    moves:
    - slide: BR
- '3':
    moves:
    - canAttack: true
      hop: FFF
- '3':
    moves:
    - leap: R
    - canAttack: true
      hop: FR
    - leap: BR
    - slide: B
    - canAttack: true
      hop: RRR
placement: 04133140 00022000
player2mirrored: true
winConditions:
  EnemyTotalPiecesLeft: 2



# Graphical interface (in progress)

In [8]:
import app #requires pygame
from threading import Thread

In [9]:
# you can play agains another human, mcts agent, or just look at two mcts agents playing
# more agent are in progress
player1agent = "USER"
player2agent = "MCTS"

In [10]:
gameThread = app.startGameThread(game, player1agent, player2agent)

In [14]:
app.theApp.game_state.printState() # can interact with game state, mid playout

Player: 1
\ 0 1 2 3 4 5 6 7
0| :1:1:1:1:1:1: :|
1| : : :1:1: : : :|
2| : : : : : : : :|
3| : : :2:2: : : :|
4| :2:2:2:2:2:2: :|


In [1]:
gameThread.join()

NameError: name 'gameThread' is not defined